<center><h1>𝓡𝓾𝓵𝓮 𝓑𝓪𝓼𝓮𝓭 𝓶𝓪𝓽𝓬𝓱𝓲𝓷𝓰💖</h1
    \></center>

Rule based information extraction is indispensable(தவிர்க்க முடியாத) for any spacy pipeline, without using any statistical model we will extract the information from this. We will discuss how to extract words using morphological features, POS tags, regex and other spacy features as well. We will use Matcher object in spacy to extract the text using this. 

There are many types that helps to do this task, let's discuss this one by one! 

* Token-based matching
* PhraseMatcher
* EntityRuler
* Combining spaCy models and matchers


## Token-based matching

It just but using the regex we can extract the text easily. But using the regex is little complicated now, because if you want to extract any specific thing you need to mention lot's of things in the regex. But in spaCy you don't need to worry on this things, because the spaCy Matcher object gives the clean code and readable code, let's look at 

```Python 
import spacy 
from spacy.matcher import Matcher  # importing matcher class

nlp = spacy.load('en_core_web_md') 
doc = nlp("Good morning, I want to reserve a ticket.")

matcher = Matcher(nlp.vocab)  # initialized the vocabulary for the matcher object (this is the usual way to do this) 
pattern = [[{"LOWER": "good"}, {"LOWER": "morning"}, {"IS_PUNCT": True}]]

matcher.add("morningGreeting",pattern)  # "morningGreeting" -> is name of this rule

matches = matcher(doc) 

for match_id, start, end in matches:  # the maches object gives triplets, so we are extracting that
    m_span = doc[start:end] 
    print(start, end, m_span.text)
```

In [1]:
import spacy 
from spacy.matcher import Matcher  

nlp = spacy.load('en_core_web_md') 
doc = nlp("Good morning, I want to reserve a ticket.")

matcher = Matcher(nlp.vocab) 
pattern = [[{"LOWER": "good"}, {"LOWER": "morning"}, {"IS_PUNCT": True}]]

matcher.add("morningGreeting",pattern) 

matches = matcher(doc) 

for match_id, start, end in matches:
    m_span = doc[start:end] 
    print(start, end, m_span.text)

0 3 Good morning,


**To get better results for this notebook, just restart and run for each cell, coz we are tagging patterns to same matcher so it will lead lot's of outputs**

In [2]:
# let's add more patterns here on the same line

doc = nlp("Good morning, I want to reserve a ticket. I will then say good evening!")

matcher = Matcher(nlp.vocab) 
pattern1 = [ [{'LOWER': 'good'}, {'LOWER': 'morning'}, {'IS_PUNCT': True}] ] 
pattern2 = [ [{'LOWER': 'good'}, {'LOWER': 'evening'}, {'IS_PUNCT': True}] ] 

matcher.add('morningGreeting', pattern1) 
matcher.add('eveningGreeting', pattern2) 

matches = matcher(doc)
for match_id, start, end in matches:
    # pattern_name = doc.vocab_strings[match_id]  # helps to get the pattern name 
    m_span = doc[start:end] 
    print( start, end, m_span.text)

0 3 Good morning,
14 17 good evening!


<img src="images/all.png" width="600"/> 

WE just seen only **LOWER**, let's see some of this things

The **Orth** and **TEXT** are same, it just extracts the similar patterns, let's see

```Python 
pattern = [ {'TEXT': 'Bill'} ]
pattern - [ {'ORTH': 'Bill'} ] # both are same don't worry 
```

Let's see **LENGTH** artibute ----

In [3]:
doc = nlp("I bought a pineapple.") 

matcher = Matcher(nlp.vocab) 
pattern = [ [{'LENGTH': 1}] ]  # Length is used for specifiying the token length 

matcher.add('oneshort', pattern) 
matches = matcher(doc) 

for a, b, c in matches: 
    docs = doc[b:c] 
    print(start, end, docs.text)


14 17 I
14 17 a
14 17 .


Let's see about **IS_ALPHA**, **IS_ASCIII** and **IS_DIGIT**. This features are usefull to find the numbers in text. Let's see

In [11]:
doc1 = nlp("I met him at 2 o'clock.")
doc2 = nlp("He brought me 2 apples.")

pattern = [{"IS_DIGIT": True},{"IS_ALPHA": True}]  # is digit and is alpha -> True 

matcher.add("numberAndPlainWord", [pattern])
matcher(doc1)

matches = matcher(doc2)

for mid, start, end in matches: 
    print(start, end, doc2[start:end])

3 4 2
3 5 2 apples
5 6 .


IS_LOWER, IS_UPPER, and IS_TITLE are useful attributes for
recognizing the token's casing. IS_UPPER is True if the token is all
uppercase letters and IS_TITLE is True if the token starts with a capital
letter. IS_LOWER is True if the token is all lowercase letters

In [15]:
doc = nlp("Take me out of your SPAM list. We never asked you to contact me. If you write again we'll SUE!!!!")

pattern = [{"IS_UPPER": True}]
matcher.add("capitals", [pattern])
matches = matcher(doc)

for mid, start, end in matches:
    print(start, end, doc[start:end])


5 6 SPAM
7 8 .
15 16 .
22 23 SUE
23 24 !
24 25 !
25 26 !
26 27 !


IS_PUNCT, IS_SPACE, and IS_STOP are usually used in patterns that
include some helper tokens and correspond to punctuation, space, and
stopword tokens (stopwords are common words of a language that do not
carry much information, such as a, an, and the in English).
IS_SENT_START is another useful attribute; it matches sentence start
tokens. Here's a pattern for sentences that start with can and the second word
has a capitalized first letter

In [18]:
doc1 = nlp("Can you swim?")
doc2 = nlp("Can Sally swim?")

pattern = [{"IS_SENT_START": True, "LOWER": "can"},{"IS_TITLE": True}]
matcher.add("canThenCapitalized", [pattern])
matcher(doc)

matches = matcher(doc2)

mid, start, end = matches[0]
print(start, end, doc2[start:end])

0 2 Can Sally


In [46]:
doc = nlp("Will you go there?")
pattern = [{"IS_SENT_START": True, "TAG": "MD"}]  # look at chapter 3 for this MD model auxilary verbs. 
matcher.add('summa', [pattern])
matches = matcher(doc)

mid, start, end = matches[0]
print(start, end, doc[start:end])


0 1 Will


In [39]:
NAME_PATTERN = [[{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}]]

doc = nlp("My name is Aravindan R ") 

matcher.add('names', NAME_PATTERN) 
matches = matcher(doc) 

for mid, start, end in matches: 
    d = doc[start:end]
    print(d.text)

Aravindan R
R
R


**See our custom extraction, this is how we need to do the extraction**

In [24]:
import spacy 
from spacy.matcher import Matcher 
matcher = Matcher(nlp.vocab) 

pattern = [{'LOWER': 'aravindan'}, {'IS_PUNCT': True}, {'LOWER': 'loves'}]

matcher.add('summa', [pattern]) 

doc = nlp('aravindan ! loves') 

matches = matcher(doc)
for ids, start, end in matches: 
    print(doc[start:end])
    print(start)
    print('ok')

aravindan ! loves
0
ok


### Regex Support for token based matching

In the first example we just just two differnt patterns in two different lines, instead of writing two different lines, we can use **or**, **in** like python let's see... 

<img src="images/comp.png" width="600"/> 

In [4]:
import spacy 
from spacy.matcher import Matcher

nlp = spacy.load('en_core_web_md')
matcher = Matcher(nlp.vocab)
doc = nlp("Good morning, I'm here. I'll say good evening!!") 

pattern = [[ 
    {'LOWER': 'good'}, 
    {'LOWER': {'IN': ['morning', 'evening']}}, 
    {'IS_PUNCT': True} 
]]

matcher.add('in', pattern) 
matches = matcher(doc) 

for i, start, end in matches: 
    print(start,end, doc[start:end])

0 3 Good morning,
10 13 good evening!


The **Comparison Operator** are usually go with **lenght**. 

In [5]:
doc = nlp("I suffered from Trichotillomania when I was in college. The doctor prescribed me Psychosomatic medicine.")
pattern = [{"LENGTH": {">=" : 10}}]

matcher.add("longWords", [pattern])
matches = matcher(doc)

for mid, start, end in matches:
    print(start, end, doc[start:end])


3 4 Trichotillomania
12 13 prescribed
14 15 Psychosomatic


### Operators

In the begining of this chapter, we discussed that, the regex gives more nasty codes that why we are moving to spacy. But using some regex function, we will explore spacy more and it will easy our tasks. 

<img src="images/regex.png" width="600"/> 

This are all the kind of operators right, we will use this operators in our specific tasks

In [5]:
# '?' operator 

import spacy 
from spacy.matcher import Matcher

nlp = spacy.load('en_core_web_md')
matcher = Matcher(nlp.vocab)

doc1 = nlp("Barack Obama visited France.")
doc2 = nlp("Barack Hussein Obama visited France.")

pattern = [[ 
    {'LOWER': 'barack'}, 
    {'LOWER': 'hussein', 'OP': '?'},  
    {'LOWER': 'obama'} 
]] 

matcher.add('bro', pattern)   
matches = matcher(doc2)   # change doc2 to doc1

for id, start, end in matches: 
    print(doc2[start:end])  # change doc2 to doc1

Barack Hussein Obama


In [32]:
NAME_PATTERN = [[   
    {'POS': 'PROPN'},
    {'POS': 'PROPN', 'OP': '?'}, 
    {'POS': 'PROPN', 'OP': '?'}
]]

doc = nlp("My name is aravindan ravi") 

matcher.add('names', NAME_PATTERN) 
matches = matcher(doc) 


mid, start, end = matches[1]  # change change 1 or 2 
print(doc[start:end])

aravindan ravi


In [41]:
# * operator (0 or more) 

doc1 = nlp("Hello hello hello, how are you?")
doc2 = nlp("Hello, how are you?")
doc3 = nlp("How are you?")

pattern = [
    {"LOWER": {"IN": ["hello", "hi", "hallo"]}, "OP": "*"}, {"IS_PUNCT": True}  # you can change * or + 
]
matcher.add("greetings", [pattern])
for mid, start, end in matcher(doc1):
    print(start, end, doc1[start:end])

0 4 Hello hello hello,
1 4 hello hello,
2 4 hello,
3 4 ,
7 8 ?


### Using POS

Using POS with the matcher, using pos with matcher helps a lot and it will helps a lot. let's see

In [43]:
doc = nlp("My name is Alice and his name was Elliot.")

pattern = [{"LOWER": "name"},{"LEMMA": "be"},{}]  # lemma: be -> it captures is, was, be tokens 
                                                  # {} -> it means wildcard, it captures after 1 token (it may be anything) 
matcher.add("pickName", [pattern])
for mid, start, end in matcher(doc):
    print(start, end, doc[start:end])


1 4 name is Alice
3 4 Alice
6 9 name was Elliot
8 9 Elliot
9 10 .


You can use the wildcard token, when you want to ignore the token or you don't know what specific token it may be: 

In [47]:
doc1 = nlp("I forwarded his email to you.")
doc2 = nlp("I forwarded an email to you.")
doc3 = nlp("I forwarded the email to you.")

pattern = [{"LEMMA": "forward"}, {}, {"LOWER": "email"}]   # now we are including any token that comes inside the two tokens
matcher.add("forwardMail", [pattern])

for mid, start, end in matcher(doc1):  
    print(start, end, doc1[start:end])
    
for mid, start, end in matcher(doc2):  # try to change 3 
    print(start, end, doc2[start:end])


1 4 forwarded his email
6 7 .
1 4 forwarded an email
6 7 .


### POS with regex 

In [50]:
doc1 = nlp("I travelled by bus.")
doc2 = nlp("She traveled by bike.")

pattern = [{"POS": "PRON"}, {"TEXT": {"REGEX": "[Tt]ravell?ed"}}]  # you used PRON -> possesive pro-noun 
matcher.add('bro', [pattern])

for mid, start, end in matcher(doc1):
    print(start, end, doc1[start:end])

for mid, start, end in matcher(doc2):
    print(start, end, doc2[start:end])

0 2 I travelled
4 5 .
0 2 She traveled
4 5 .


```Python 
pattern = [{"POS": "PRON"}, {"TEXT": {"REGEX": "[Tt]ravell?ed"}}]  # [Tt] -> the first word will be "T" or 't'
matcher.add('bro', [pattern])                                      # ravell? -> it just a optinal one  
                                                                   # ed -> compulsary one 
```

In [4]:
# Here you are using the verbs 

import spacy 
from spacy.matcher import Matcher

nlp = spacy.load('en_core_web_md')
matcher = Matcher(nlp.vocab)

doc = nlp("I went to Italy; he has been there too. His mother also has told me she wants to visit Rome.")

pattern = [{"TAG": {"REGEX": "^V"}}]  # verbs dude
matcher.add("verbs", [pattern])

for mid, start, end in matcher(doc):
    print(start, end, doc[start:end])

1 2 went
6 7 has
7 8 been
14 15 has
15 16 told
18 19 wants
20 21 visit


```Python
pattern = [{"TAG": {"REGEX": "^V"}}]  # ^V -> (includes all) VB, VGD, VBG, VBN, VBP, VBZ
```

If you have any doubt in regex you can use this [**website**](https://regex101.com/) to know what happening inside the regex

Same like, if you have any doubt in matcher you can visit this [**website**](https://explosion.ai/demos/matcher) to know what happening inside this. 

## Phrase Matcher

While processing financial, medical, or legal text, often we have long lists
and dictionaries and we want to scan the text against our lists. As we saw in
the previous section, Matcher patterns are quite handcrafted; we coded each
token individually. If you have a long list of phrases, Matcher is not very
handy. It's not possible to code all the terms one by one.
spaCy offers a solution for comparing text against long dictionaries – the
PhraseMatcher class. The PhraseMatcher class helps us match long
dictionaries.

```Python 
import spacy
from spacy.matcher import PhraseMatcher  # phrase matcher class

nlp = spacy.load("en_core_web_md")
matcher = PhraseMatcher(nlp.vocab)  # initialized phrase matcher object with nlp.vocab

terms = ["Angela Merkel", "Donald Trump", "Alexis Tsipras"]
patterns = [nlp.make_doc(term) for term in terms]  # make_doc helps to create one by one patterns, In another term it will create a doc for individual word and acts like a tokenizer.

matcher.add("politiciansList", None, *patterns)  

doc = nlp("3 EU leaders met in Berlin. German chancellor Angela Merkel first welcomed the US president Donald Trump. The following day Alexis Tsipras joined them in Brandenburg.")

matches = matcher(doc)
for mid, start, end in matches:
    print(start, end, doc[start:end])
```

In [2]:
import spacy
from spacy.matcher import PhraseMatcher

nlp = spacy.load("en_core_web_md")
matcher = PhraseMatcher(nlp.vocab)

terms = ["Angela Merkel", "Donald Trump", "Alexis Tsipras"]
patterns = [nlp.make_doc(term) for term in terms]
matcher.add("politiciansList", None, *patterns)

doc = nlp("3 EU leaders met in Berlin. German chancellor Angela Merkel first welcomed the US president Donald Trump. The following day Alexis Tsipras joined them in Brandenburg.")
matches = matcher(doc)

for mid, start, end in matches:
    print(start, end, doc[start:end])

9 11 Angela Merkel
16 18 Donald Trump
22 24 Alexis Tsipras


Using Attributes

In [7]:
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")  # Lower attribute

terms = ["Asset", "Investment", "Derivatives", "Demand", "Market"]
patterns = [nlp.make_doc(i) for i in terms] 

matcher.add('Summa', None, *patterns)

doc = nlp("During the last decade, derivatives market became an asset class of their own and influenced the financial landscape strongly.")
matches = matcher(doc) 

for id, start, end in matches: 
    print(start, end, doc[start:end])

5 6 derivatives
6 7 market
9 10 asset


If you want to extract the system logs, IP numbers, dates and other numerical values, we will use the Matcher object with shape, it helps to extract the information from the text. 

In [15]:
matcher = PhraseMatcher(nlp.vocab, attr="SHAPE")  # shape attribute

ip_nums = ["127.0.0.0", "127.256.0.0"]
patterns = [nlp.make_doc(ip) for ip in ip_nums]
matcher.add("IPNums", None, *patterns)

doc = nlp("This log contains the following IP addresses: 192.1.1.1 and 192.12.1.1 and 192.160.1.1 .")
for mid, start, end in matcher(doc):
    print(start, end, doc[start:end])

8 9 192.1.1.1
12 13 192.160.1.1


## Entity Ruler

We know entity is used to map person, place, things. The awesome thing is that, we can extract the person, name (entities) using this entity ruler. 

In [29]:
pattern = [{"ENT_TYPE": "PERSON"}]  # we are mentioning entity type as person, let's see the magic 

matcher.add("personEnt", [pattern]) 

doc = nlp("Aravind loves Bill Gates too much")

matches = matcher(doc) 

for id_, start, end in matches: 
    print(doc[start:end]) 

Aravind
Bill
Gates


Here we got indivial names, if you want to extract full names, you can use **Operator** in the code

In [32]:
pattern = [{"ENT_TYPE": "PERSON", "OP":"+"}]  # we are mentioning entity type as person, let's see the magic 

matcher.add("personEnt", [pattern]) 

doc = nlp("Aravind Ravi loves Bill Gates too much")

matches = matcher(doc) 

for id_, start, end in matches: 
    print(doc[start:end]) 

Aravind
Ravi
Aravind Ravi
Bill
Gates
Bill Gates


Usually, we do this with lots of operators and pos (I mean, we extract lots of information in a line), let's see

In [37]:
import spacy 
from spacy.matcher import Matcher 

matcher = Matcher(nlp.vocab)

pattern = [[
    {"ENT_TYPE":"PERSON", "OP":"+"}, 
    {"POS":"VERB"} 
]]

matcher.add('podu', pattern) 
doc = nlp("Today German chancellor Angela Merkel met with the US president.")

matches = matcher(doc) 
for mid, start, end in matches: 
    print(start, end, doc[start:end])
    

4 6 Merkel met
3 6 Angela Merkel met


See, we get the verb after pron, 

## Combining  spaCy models and matchers

Let's talk about the domain releated problems. like you want to extract the greek names from the wikipedia aritcle, our normal spacy model fails to do that, because spaCy trained in english words so it don't extract the  words, like this cases you can use the entity ruler. 

The entity ruler is not a class, it's just a pipeline, like if you want to add new to entites in the pipeline, you can add the ent using this entity ruler. 

In [20]:
import spacy
nlp = spacy.load("en_core_web_md")

text = "The village of Treblinka is in Poland. Treblinka was also an extermination camp."
ruler = nlp.add_pipe("entity_ruler")  # add entity ruler to our spacy pipeline 

patterns = [{"label": "GPE", "pattern": "Treblinka"}]
ruler.add_patterns(patterns)  # add patterns to the entity ruler pipeline 
doc = nlp(text)

for ent in doc.ents:
    print (ent.text, ent.label_)

Treblinka GPE
Poland GPE
Treblinka GPE


In [23]:
import spacy 
nlp = spacy.load('en_core_web_md')

doc = nlp("I have an acccount with chime since 2017")
print(doc.ents)
print('see you have only date here', 'bellow this you have a chime, it was added by entity ruler')

ruler = nlp.add_pipe("entity_ruler")  # create pipeline 

patterns = [{"label": "ORG", "pattern": [{"LOWER": "chime"}]}]  # create patterns 
ruler.add_patterns(patterns)  # add your patterns

doc = nlp("I have an acccount with chime since 2017")
for ent in doc.ents: 
    print (ent.text, ent.label_)

(2017,)
see you have only date here bellow this you have a chime, it was added by entity ruler
chime ORG
2017 DATE


This is really an amazing features, we can combine both the matcher and entity ruler to do lot's of things, let's go for real world examples and parse out the IBAN numbers in the list. IBAN numbers are just a finanical numbers that looks like this `BE71 0961 2345 6769`, `BE71 ME61 2345 6769`

We can able to say that, the IBAN numbers are starting with two caps letters following by the two numbers, so we use `{'SHAPE':"XXdd"}` Here the XX-> represents two capital letters and dd-> represents two numbers. 

Then we have sequence of 4 numbers for three blocks and sometiems the second block number is starting from the two caps digit so, we use regular expressiion for this  `\d{1,4}` - > This pattern will match the digits block

In [28]:
import spacy 
import en_core_web_md
from spacy.matcher import Matcher
nlp = en_core_web_md.load()
matcher = Matcher(nlp.vocab)

pattern = [[ 
    {"SHAPE": "XXdd"}, 
    {"TEXT": {"REGEX": "\d{1,4}"}, "OP":"+"}
]]

doc1 = nlp("My IBAN number is BE71 0961 2345 6769, please send the money there.")
doc2 = nlp("My IBAN number is FR76 BE00 6000 0112 3456 7890 189 please send the money there.")  # STARTING WITH DIGITS 

matcher.add('summa', pattern) 
matches1 = matcher(doc1)
matches2 = matcher(doc2)

for id_, start, end in matches1:
    print(doc1[start:end]) 
    
for id_, start, end in matches2:
    print(doc2[start:end]) 


BE71 0961
BE71 0961 2345
BE71 0961 2345 6769
FR76 BE00
FR76 BE00 6000
BE00 6000
FR76 BE00 6000 0112
BE00 6000 0112
FR76 BE00 6000 0112 3456
BE00 6000 0112 3456
FR76 BE00 6000 0112 3456 7890
BE00 6000 0112 3456 7890
FR76 BE00 6000 0112 3456 7890 189
BE00 6000 0112 3456 7890 189


let's do how to extract the phone numbers. 

Let's start with the US phone number format. A US number is written as (541)
754-3010 domestically or +1 (541) 754-3010 internationally. We can form
our pattern with an optional +1, then a three-digit area code, then two blocks
of numbers separated with an optional -. 

Here is the pattern:

```{"TEXT": "+1", "OP": "?"}, {"TEXT": "("}, {"SHAPE": "ddd"},
{"TEXT": ")"}, {"SHAPE": "ddd"}, {"TEXT": "-", "OP": "?"},
{"SHAPE": "dddd"}```

In [29]:
import spacy 
from spacy.matcher import Matcher 

nlp = en_core_web_md.load()
matcher = Matcher(nlp.vocab) 

pattern = [[ 
    {"TEXT": "+1", "OP": "?"}, {"TEXT": "("},
    {"SHAPE": "ddd"}, {"TEXT": ")"},
    {"SHAPE": "ddd"}, {"TEXT": "-", "OP": "?"},
    {"SHAPE": "dddd"}
]] 

doc1 = nlp("You can call my office on +1 (221) 102-2423 or email me directly.")
doc2 = nlp("You can call me on (221) 102 2423 or text me.")

matcher.add('summa1', pattern) 

matches1 = matcher(doc1) 
matches2 = matcher(doc2) 

for id_, start, end in matches1: 
    print(doc1[start:end])
    
for id_, start, end in matches2: 
    print(doc2[start:end])

+1 (221) 102-2423
(221) 102-2423
(221) 102 2423


Let's try with `indian phone number`. The phone number looks like this `+91 9786420128`, `9786420128`

In [39]:
pattern = [[ 
    {"TEXT":"+91", "OP":"?"}, 
    {"TEXT": {"REGEX":"\d{10}"}, "OP":"+"}
]]

matcher.add('summa2', pattern) 
doc1 = nlp("You can call my office on +91 9786420128 or email me directly.")

matches1 = matcher(doc1) 

for id_, start, end in matches1: 
    print(doc1[start:end])

+91 9786420128
9786420128


Let's extract the mentions, what I mean is nothing but extracting the  companies are mentioned in which ways. by using this small dataset

```Markdown
CafeA is very generous with the portions.
CafeB is horrible, we waited for mins for a table.
RestaurantA is terribly expensive, stay away!
RestaurantB is pretty amazing, we recommend.
```

What we're looking for is most probably patterns of the BusinessName
is/was/be adverb* adjective form. The following pattern would work:

`pattern = [{"ENT_TYPE": "ORG"}, {"LEMMA": "be"}, {"POS": "ADV", "OP": "*"}, {"POS": "ADJ"}]`


In [50]:
import spacy 
from spacy.matcher import Matcher 

nlp = en_core_web_md.load()
matcher = Matcher(nlp.vocab) 

pattern = [{"LOWER": "acme"}, {"LEMMA": "be"}, {"POS": "ADV", "OP": "*"}, {"POS": "ADJ"}]


doc = nlp("CafeA is very generous with the portions. CafeB is horrible, we waited for mins for a table. RestaurantA is terribly expensive, stay away! RestaurantB is pretty amazing, we recommend.")

matcher.add('summa3', [pattern]) 

matches = matcher(doc) 

for id_, start, end in matches: 
    print(doc[start:end])
print(matches)
print('k')

[]
k


I don't know why we are not getting this. 

**Let's match hastag**

Hastag has no space and '#" after this there is no space,so we can get easily

In [52]:
doc = nlp("Start working out now #WeekendShred")

pattern = [{"TEXT": "#"}, {"IS_ASCII": True}]
matcher = Matcher(nlp.vocab)
matcher.add("hashTag", [pattern])
matches = matcher(doc)

for mid, start, end in matches:
    print(start, end, doc[start:end])


4 6 #WeekendShred


**Let's match emoji**

An emoji is usually coded with lists according to their
sentiment value, such as positive, negative, happy, sad, and so on. Here,
we'll separate emojis into two classes, positive and negative

In [62]:
import spacy 
from spacy.matcher import Matcher 

nlp = en_core_web_md.load()
matcher = Matcher(nlp.vocab)

pos_emoji = ["😆", "🙂", "😎"]
neg_emoji = ["😭", "💔", "😿"]


pos_patterns = [[{"ORTH": emoji}] for emoji in pos_emoji]
neg_patterns = [[{"ORTH": emoji}] for emoji in neg_emoji]

matcher.add("posEmoji", pos_patterns)
matcher.add("negEmoji", neg_patterns)


doc = nlp(" I love Zara 😿")
for mid, start, end in matcher(doc):
    print(start, end, doc[start:end])
    # print(doc[start:end].name_)  # Here I need to get the pattern name 


4 5 😿


In [65]:
# let's extract mr woman and mr men or prof doctor 

import spacy 
nlp = en_core_web_md.load()

patterns = [{"label": "TITLE", "pattern": [{"LOWER": {"IN": ["ms.", "mr.", "mrs.", "prof.", "dr."]}}]}]
ruler = nlp.add_pipe("entity_ruler")
ruler.add_patterns(patterns)

doc = nlp("Ms. Smith left her house")
print([(ent.text, ent.label_) for ent in doc.ents])

[('Ms.', 'TITLE'), ('Smith', 'PERSON')]
